# Import 2 DataSets, clean and merge

In [29]:
import pandas as pd

## Import the main dataset (EUMA)

In [30]:
# import the main dataset
euma = pd.read_excel('.\EUMA.xlsx')

In [31]:
euma.head(5)

,Entity,Credit Controller,Mc,Invoice Date,Invoice Type,Euro SUM,Curr,Due Date,Billing Account Number
0,GUT,Ritgi Dabey,SSM62,13-AUG-21,INV,724.446510,USD,2021-09-12,0000001258-9300903-1-GUT
1,GHD,Viaty Lia,SSM31,15-JUL-21,INV,4774.667159,HKD,2021-08-14,0000001352-ERS05019HK-GHD
2,GUT,George Csakravay,SSM62,13-AUG-21,INV,7541.946932,USD,2021-09-27,0000001508-9300749-10-GUT
3,GUT,George Csakravay,SSM62,13-AUG-21,INV,26937.147202,USD,2021-09-27,0000001508-9300749-11-GUT
4,GUT,George Csakravay,SSM62,13-AUG-21,INV,62010.765589,USD,2021-09-27,0000001508-9300749-14-GUT


In [32]:
euma.columns

Index(['Entity', 'Credit Controller', 'Mc', 'Invoice Date', 'Invoice Type',
       ' Euro SUM', 'Curr', 'Due Date', 'Billing Account Number'],
      dtype='object')

In [33]:
euma['Credit Controller'].unique()[:10]

array(['Ritgi Dabey', 'Viaty Lia', 'George Csakravay', 'Giacomo Gigante',
       'Max Koziola', 'Shwatha Ravain', 'Ul Hinder', 'Same Jebasa',
       'Leama Rosemar', 'Ils Pandurangana'], dtype=object)

In [34]:
# Remove rows where Credit Controller is "Max Koziola"
euma=euma[euma['Credit Controller']!='Max Koziola']

In [35]:
# Add column 'BAN' as an extraction from column 'Billing Account Number'
def extract_ban(composed_ban):
    len_ban = len(composed_ban)-15
    ban = composed_ban[11 : 11+len_ban]
    return ban

euma['BAN'] = list(map(extract_ban, euma['Billing Account Number']))
euma.head(3)   

,Entity,Credit Controller,Mc,Invoice Date,Invoice Type,Euro SUM,Curr,Due Date,Billing Account Number,BAN
0,GUT,Ritgi Dabey,SSM62,13-AUG-21,INV,724.446510,USD,2021-09-12,0000001258-9300903-1-GUT,9300903-1
1,GHD,Viaty Lia,SSM31,15-JUL-21,INV,4774.667159,HKD,2021-08-14,0000001352-ERS05019HK-GHD,ERS05019HK
2,GUT,George Csakravay,SSM62,13-AUG-21,INV,7541.946932,USD,2021-09-27,0000001508-9300749-10-GUT,9300749-10


## Import second file containing a column to merge

In [36]:
# Import GLCODEs
gldata = pd.read_excel('.\TABLE_IMS_FINANCE_PARTNERS 12082021.xlsx')

In [37]:
gldata.head()

,Unnamed: 0,DEFAULT (customer number must be empty),Unnamed: 2,0,Other customers,114180,Unnamed: 6
0,*,DEFAULT (customer number must be empty),NaN,0,Other customers,114180,NaN
1,MUUS01,NaN,36869.0,ESI,SITA,214315,GUT
2,MUUS02,NaN,36869.0,ESI,SITA,214315,GUT
3,MUUS03,NaN,36869.0,ESI,SITA,214315,GUT
4,MUUS04,NaN,36869.0,ESI,SITA,214315,GUT


In [38]:
# I need only columns 'Unnamed: 0' and 114180. First row on top unnecessary
gldata=gldata.drop(0, axis=0).rename(columns={'Unnamed: 0':'BAN',114180:'GL_CODE'})[['BAN', 'GL_CODE']]

In [39]:
gldata.head(3)

,BAN,GL_CODE
1,MUUS01,214315
2,MUUS02,214315
3,MUUS03,214315


In [40]:
# Get rid of 'na' BANs (not providing any info)
gldata=gldata[gldata['BAN'].notna()]

In [41]:
# check if there are some other missing values
gldata.isna().sum()

BAN        0
GL_CODE    0
dtype: int64

In [42]:
gldata.dtypes

BAN        object
GL_CODE     int64
dtype: object

In [43]:
# I want column GL_CODE to be type string
gldata['GL_CODE']=gldata['GL_CODE'].astype(str)
gldata.dtypes

BAN        object
GL_CODE    object
dtype: object

## Join the datasets

In [44]:
df = pd.merge(euma, gldata, how='left', on='BAN')

In [45]:
df.head()

,Entity,Credit Controller,Mc,Invoice Date,Invoice Type,Euro SUM,Curr,Due Date,Billing Account Number,BAN,GL_CODE
0,GUT,Ritgi Dabey,SSM62,13-AUG-21,INV,724.446510,USD,2021-09-12,0000001258-9300903-1-GUT,9300903-1,NaN
1,GHD,Viaty Lia,SSM31,15-JUL-21,INV,4774.667159,HKD,2021-08-14,0000001352-ERS05019HK-GHD,ERS05019HK,NaN
2,GUT,George Csakravay,SSM62,13-AUG-21,INV,7541.946932,USD,2021-09-27,0000001508-9300749-10-GUT,9300749-10,NaN
3,GUT,George Csakravay,SSM62,13-AUG-21,INV,26937.147202,USD,2021-09-27,0000001508-9300749-11-GUT,9300749-11,NaN
4,GUT,George Csakravay,SSM62,13-AUG-21,INV,62010.765589,USD,2021-09-27,0000001508-9300749-14-GUT,9300749-14,NaN


In [46]:
# create a new df filtered on 'GL_CODE' not na
df_with_codes = df[df['GL_CODE'].notna()].head().reset_index(drop=True)
df_with_codes.head(4)

,Entity,Credit Controller,Mc,Invoice Date,Invoice Type,Euro SUM,Curr,Due Date,Billing Account Number,BAN,GL_CODE
0,GCH,Marc Mozolsky,SSX01,16-JUL-21,INV,71629.441640,CLP,2021-08-30,0000010748-GSTA18-GCH,GSTA18,214315
1,GCH,Marc Mozolsky,SSX01,18-AUG-21,CM,-14561.410706,CLP,2021-08-18,0000010748-GSTA18-GCH,GSTA18,214315
2,GCH,Marc Mozolsky,SSX01,18-AUG-21,INV,2.503291,CLP,2021-09-30,0000010748-GSTA18-GCH,GSTA18,214315
3,GCH,Marc Mozolsky,SSX01,18-AUG-21,INV,55062.419920,CLP,2021-09-30,0000010748-GSTA18-GCH,GSTA18,214315
